<a href="https://colab.research.google.com/github/Englishson0909/2024spring/blob/main/Corpus/SentimentalAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#🌿 NLP and Digital Humanities
## Part 1. Natural Language Processing
## Part 2. Topic-Modeling - Presidential Address
## **Part 3. Sentiment Analysis - English Literature (e.g., Harry Potter)**
## Part 4. Clustering Analysis - movies

학교에서 쓰는 글에 긍정적인 표현/긍정적인 표현으로 확인 다른그룹에 비해 어떤가? 교과서 감성분석 중립적일 수도 있으니 긍정적이 것을 위해 긍정적일도

---
# Part III. Sentiment Analysis (w/ Harry Potter)
---

In [ ]:
#@markdown + Intro to Sentimental Analysis

from IPython.display import display
import ipywidgets as widgets
import requests

def on_button_click(button):
    sn = int(button.description) - 1
    image.value = requests.get(urls[sn]).content

urls = ["https://raw.githubusercontent.com/junkyuhufs/workshop/main/slide.13.png",
        "https://raw.githubusercontent.com/junkyuhufs/workshop/main/slide.12.png",
        "https://raw.githubusercontent.com/junkyuhufs/workshop/main/slide.14.png"
]

button_layout = widgets.Layout(width='50px', height='30px')

buttons = [widgets.Button(description=str(i), layout=button_layout) for i in range(1, 4)]
for button in buttons:
    button.on_click(on_button_click)

image = widgets.Image(value=requests.get(urls[0]).content, width="800", height="600")

display(widgets.HBox([image, widgets.VBox(buttons)]))

## Preparing the data to analyze

+ [Source link: Harry Potter sharing Github site](https://github.com/ErikaJacobs/Harry-Potter-Text-Mining.git)

+ [Text example](https://raw.githubusercontent.com/ErikaJacobs/Harry-Potter-Text-Mining/master/Book%20Text/HPBook1.txt)

In [ ]:
#@markdown + Bring the text: Harry Potter
!git clone https://github.com/ErikaJacobs/Harry-Potter-Text-Mining.git

In [ ]:
#@markdown + Text pre-processing: data processing using {pandas} (One chapter in one cell)
import pandas as pd #Importing Pandas package
%cd /content/Harry-Potter-Text-Mining/Book Text

import glob
fns = glob.glob('*.txt')
df = pd.DataFrame()
for fn in fns:
  dftmp = pd.read_csv(fn, sep="@")
  df = pd.concat([df, dftmp])

%cd /content

df

In [ ]:
#@markdown + Remove stop words
import nltk #Import NLTK library
nltk.download('stopwords')
nltk.download('punkt') #installed punkt to fix error
from nltk import word_tokenize
from nltk.corpus import stopwords #Import stopwords to Python

stopwords = set(stopwords.words('english')) #English stopwords assigned to "stopwords" object

import string #Punctuation

# Function for removing punctuation
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

stopwords = [''.join(item for item in x if item not in string.punctuation) for x in stopwords] #Remove punctuation from stopwords

df['WordCountText']=df['Text'].str.lower().apply(remove_punctuations).apply(word_tokenize) # Word Count Text
# Word Count
df['WordCloudText']=df['WordCountText'].apply(lambda x: [word for word in x if word not in stopwords]) # Word Cloud Text
df['WordCount'] = df['WordCountText'].str.len() #Word Count Per Chapter

In [ ]:
#@markdown + Book chapter to Sentences
# Creating a table breaking down the text by each sentence, rather than each chapter.
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk.stem import PorterStemmer

# Make smaller table - reset index to prepare for further work
dfsentiment = df[['Book','Chapter','Text']].reset_index() \
    .drop(["index"], axis=1)
dfsentiment = dfsentiment.join(dfsentiment.Text.apply(sent_tokenize).rename('Sentences')) # Breaking apart text into sentences

#Put every tokenized sentence into its own row
dfsentiment2 = dfsentiment.Sentences.apply(pd.Series) \
    .merge(dfsentiment, left_index = True, right_index = True) \
    .drop(["Text"], axis = 1) \
    .drop(["Sentences"], axis = 1) \
    .melt(id_vars = ['Book', 'Chapter'], value_name = "Sentence") \
    .drop("variable", axis = 1) \
    .dropna()

# Sort new table by Book and Chapter - reset index to reflect new order
dfsentiment2=dfsentiment2.sort_values(by=['Book', 'Chapter']) \
    .reset_index() \
    .drop(['index'], axis = 1)

# Clean punctuation, lower case
dfsentiment2['Sentence']=dfsentiment2.Sentence.apply(remove_punctuations).apply(lambda x: x.lower()) \

# Check first five values
dfsentiment2

## Proceed to a Sentimental Analysis

In [ ]:
#@markdown Install and import {VADER} library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
sid=nltk.sentiment.vader.SentimentIntensityAnalyzer()

###Harry Potter example texts (stemmed texts)


|출처 | 예시문장 | 감정 |
|--|--|--|
|[0,1,1]|'the boy who lived mr and mrs dursley of number four privet drive were **proud** to say that they were perfectly normal **thank** you very much'|😄 Positive|[0,1,1]|
|[1,1,1]|'they were the last people youd expect to be involved in anything **strange** or **mysterious** because they just didnt hold with such **nonsense**'|😡 Negative |
|[2,1,1]|'mr dursley was the director of a firm called grunnings which made drills'|😐 Neutral|


+ *Note*. 출처 == [sentence number, Book, Chapter]

In [ ]:
#@markdown + Assign sentiment scores to sentences; Compound, positive, negative, neutral
# Get intensity scores of each sentence
dfsentiment2['Score']=dfsentiment2.Sentence.apply(lambda x: sid.polarity_scores(x))

# Place scores in own columns
dfsentiment2['CompScore']=dfsentiment2.Score.apply(lambda x: x.get("compound"))
dfsentiment2['PosScore']=dfsentiment2.Score.apply(lambda x: x.get("pos"))
dfsentiment2['NegScore']=dfsentiment2.Score.apply(lambda x: x.get("neg"))
dfsentiment2['NeuScore']=dfsentiment2.Score.apply(lambda x: x.get("neu"))

# With scores extracted, the original score field can be removed
dfsentiment2 = dfsentiment2.drop(["Score"], axis=1)

# Adding Sentiment Flags
dfsentiment2['PosFlag'] = dfsentiment2.CompScore.apply(lambda x: 1 if x >= 0.05 else 0)
dfsentiment2['NegFlag'] = dfsentiment2.CompScore.apply(lambda x: 1 if x <= -0.05 else 0)
dfsentiment2['NeuFlag'] = dfsentiment2.CompScore.apply(lambda x: 1 if x < 0.05 and x > -0.05 else 0)

In sentiment analysis, the term **"compound"** typically refers to a composite score that is used to represent the overall sentiment of a sentence, combining the individual positive, negative, and neutral scores into a single measure. This score is designed to indicate the overall emotional tone of the text, whether it's positive, negative, or neutral.

In [ ]:
dfsentiment2.head(20)

In [ ]:
#@markdown [1] Sentimental Analysis result (barplot): Neg, Neu, Pos

print('* Negative Flag: ', dfsentiment2['NegFlag'].sum())
print('* Neutral Flag: ', dfsentiment2['NeuFlag'].sum())
print('* Positive Flag: ', dfsentiment2['PosFlag'].sum())
print("="*50)
print('Total: ',dfsentiment2['PosFlag'].sum()+dfsentiment2['NeuFlag'].sum()+dfsentiment2['NegFlag'].sum())


import numpy as np
import matplotlib.pyplot as plt

Negative = int(dfsentiment2['NegFlag'].sum())
Neutral = int(dfsentiment2['NeuFlag'].sum())
Positive = int(dfsentiment2['PosFlag'].sum())

# Your three integer frequencies
freqs = [Negative, Neutral, Positive]
# freqs = [18385, 33544, 19055]

# Create labels for the bars
labels = ['Negative', 'Neutral', 'Positive']

# Create x coordinates for the bars
x = np.arange(len(labels))

# Generate the bar plot
plt.bar(x, freqs)


# Specify the colors for each category
colors = ['lightblue', 'gray', 'orange']

# Generate the bar plot with custom colors

bars = plt.bar(x, freqs, color=colors)
# Add labels to the x-axis
plt.xticks(x, labels)

# Set axis labels
plt.xlabel('Categories')
plt.ylabel('Frequency')

# Set a title for the plot
plt.title('Bar plot of Sentiment categories')
plt.ylim(0, 40000)
# Add the frequency text within each bar
for bar, freq in zip(bars, freqs):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() - 2, str(freq),
             ha='center', va='bottom', fontsize=12, color='gray')


# Display the plot
plt.show()


In [ ]:
#@markdown [2] Sentimental Analysis result (Pie chart): ratio to check
import numpy as np
import matplotlib.pyplot as plt

Negative = int(dfsentiment2['NegFlag'].sum())
Neutral = int(dfsentiment2['NeuFlag'].sum())
Positive = int(dfsentiment2['PosFlag'].sum())

# Your three integer frequencies
freqs = [Negative, Neutral, Positive]

# Create labels for the segments
labels = ['Negative', 'Neutral', 'Positive']

# Specify the colors for each segment
colors = ['lightblue', 'gray', 'orange']

# Generate the pie chart with custom colors
plt.pie(freqs, labels=labels, colors=colors, autopct='%.1f%%', startangle=90)

# Set a title for the plot
plt.title('Pie chart of Sentiment categories')

# Display the plot
plt.show()


### Time Series (시계열) Analysis and visualization

In [ ]:
#@markdown Time series (The entire chapters: 1~37)
# Assuming 'BookTitle' is correctly mapped from 'Book'
dfs_grouped = dfsentiment2.groupby(['Chapter', 'Book']).agg({
    'CompScore': 'mean',
    'PosScore': 'mean',
    'NegScore': 'mean',
    'NeuScore': 'mean'
})

# Optionally, check the output to ensure it's as expected
print(dfs_grouped.head())

# If you want to plot the results
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10, 6))
dfs_grouped['CompScore'].plot(ax=ax)
plt.ylabel('Average Compound Sentiment Score')
plt.title('Average Compound Sentiment Scores by Chapter and Book')
plt.show()


In [ ]:
#@markdown Define a custom color list for each book
colors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0', '#f032e6', '#bcf60c', '#fabebe', '#008080', '#e6beff', '#9a6324', '#fffac8', '#800000', '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080', '#ffffff', '#000000']

fig, ax = plt.subplots(figsize=(10, 6))

chapter_numbers = sorted(dfsentiment2['Chapter'].unique())
for i, chapter in enumerate(chapter_numbers):
    if chapter in dfs_grouped['CompScore']:
        ax.plot(dfs_grouped['CompScore'][chapter], label=f'Chapter {chapter}', color=colors[i % len(colors)])

plt.ylabel('Average Compound Sentiment Score')
plt.title('Average Compound Sentiment Scores by Chapter and Book')
plt.legend()
plt.show()


**ChatGPT says:** In Harry Potter and the Deathly Hallows, the main tragic event between chapters 25 and 28 is the aftermath of Dobby's death. Dobby, the house-elf, dies a hero while saving Harry and his friends from the Death Eaters at Malfoy Manor.


+ The main tragic event that occurs between chapters 25 and 28 in Harry Potter and the Deathly Hallows (Book 7) is the death of Dobby, the house-elf. This event takes place in Chapter 23, "Malfoy Manor", and is not directly within the range you mentioned (chapters 25-28). However, its consequences and emotional impact on the characters continue to resonate in the following chapters.

+ Dobby dies while helping Harry, Hermione, Ron, Luna, Dean, and Griphook escape from Malfoy Manor, where they were captured and held prisoner by Bellatrix Lestrange and other Death Eaters. In an attempt to rescue them, Dobby Apparates into the Manor, and in the ensuing chaos, he is fatally struck by a knife thrown by Bellatrix Lestrange as they Disapparate away.

+ Dobby's death is a significant and heartbreaking moment in the story, as he dies a hero, saving Harry and his friends from the Death Eaters. It underscores the themes of loyalty, sacrifice, and the importance of valuing all beings, regardless of their background or status.

**ChatGPT says:**

**Ending story:** The ending of Harry Potter and the Deathly Hallows (Book 7) sees the final defeat of Lord Voldemort by Harry Potter during the Battle of Hogwarts. The ending is generally considered happy, as it brings the closure of the long-standing conflict between Harry and Voldemort, and most of the central characters, including Harry, Ron, and Hermione, survive and go on to live fulfilling lives. The book concludes with an epilogue set 19 years later, where Harry, Ron, Hermione, and their families are shown sending their own children off to Hogwarts School of Witchcraft and Wizardry, signifying a hopeful and peaceful future.